In [ ]:
from cobra import *
from libsbml import *
import sys, getopt, os.path
from tqdm import tqdm
import zipfile

In [ ]:
# exemplary file
omexFile = "SARS-CoV-2_CoreModelsBALF1Epithelial.omex"
sbmlPath = omexFile.replace(".omex","")
flattenedPath = omexFile.replace(".omex","_flattened")

In [ ]:
# unzip OMEX file of interest
with zipfile.ZipFile(omexFile, 'r') as zip_ref:
    zip_ref.extractall(sbmlPath)

In [ ]:
# create directory to store results
os.mkdir(flattenedPath)

In [ ]:
# convert hierarchical models defined with the Hierarchical Model Composition package into a "flattened" version

for model in os.listdir(sbmlPath):
    
    sbmldoc = readSBML(sbmlPath+"/"+model)
    
    # define converter options
    props = ConversionProperties()
    props.addOption("flatten comp", True)       

    # create "flattened" version
    result = sbmldoc.convert(props)
    if result != LIBSBML_OPERATION_SUCCESS:
        sbmldoc.printErrors()
        raise SystemExit("Conversion failed... %tb")
        
    writer  = SBMLWriter()
    writer.writeSBML(sbmldoc, flattenedPath+"/"+model.replace(".sbml",".xml"))
    
    print(model, "Flattening.... OK")


In [ ]:
# read with cobrapy
for flatObj in tqdm(os.listdir(flattenedPath)):
    
    model = io.read_sbml_model(flattenedPath+"/"+flatObj)
    
    # Solve error: '' is not a valid SBML 'SId'.
    model.id = str(flatObj).replace(".xml","").replace("-","_")
    
    print(model.id, "ID correction.... OK")
    
    io.write_sbml_model(model,flattenedPath+"/"+flatObj)